# Python Advanced

*Gyuri*, `EU_edge`

## Diamond inheritance problem
`Cls.__mro__` - method resolution order

In [1]:
class A(object): pass
class B(A): pass
class C(A): pass
class D(C, B): pass

print(D.__mro__)

(<class '__main__.D'>, <class '__main__.C'>, <class '__main__.B'>, <class '__main__.A'>, <class 'object'>)


## Mixins
You can add *base classes* runtime

In [7]:
class A(object): pass
class B(object): pass
class C(A): pass

print(C.__mro__)
C.__bases__ += (B,)
print(C.__mro__)

(<class '__main__.C'>, <class '__main__.A'>, <class 'object'>)
(<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class 'object'>)


## Variable scope
- No declaration, just use any variable whenever you want
- There's `global` but ...
- Namespaces are dicts
- `module.submodule.variable = 42`
- `del` keyword
- `__builtins__`, `locals()`, `globals()`

In [10]:
def scopetest(mut, imut):
    global gvar
    gvar = 11
    lvar = 12
    mut += [2]
    imut += "pen"
    print('scopetest locals: ', locals())

gvar, lvar, amut, aimut = 1, 2, [1], "apple"
print(gvar, lvar, amut, aimut)
scopetest(amut, aimut)
print(gvar, lvar, amut, aimut)

1 2 [1] apple
scopetest locals:  {'imut': 'applepen', 'mut': [1, 2], 'lvar': 12}
11 2 [1, 2] apple


## Iterators
- `container.__iter__()` -> get the iterator
- `iterator.__iter__()` -> get the iterator (self)
- `iterator.__next__()` -> next item, or StopIteration exception

In [2]:
a = [1,2]
i = iter(a) # calls a.__iter__()
print(next(i)) # calls i.__next__()
print(next(i))
print(next(i))

for x in a:
    print(x)

1
2


StopIteration: 

## Comprehensions
The *functional way* of list/dict creation

In [23]:
print([x*2 for x in f100])
print({'%d^2' % x: x**2 for x in range(4)})

[2, 2, 4, 6, 10, 16, 26, 42, 68, 110, 178]
{'3^2': 9, '2^2': 4, '0^2': 0, '1^2': 1}


## Itertools
Useful toolset for iterations

*Some parts have been moved to the standard library meanwhile, like **zip** *

In [33]:
for x,y in zip(range(5), f100):
    print("fib(%d) = %d" % (x, y))

fib(0) = 1
fib(1) = 1
fib(2) = 2
fib(3) = 3
fib(4) = 5


In [34]:
from itertools import chain
for x in chain(range(0,5), range(5,10)):
    print(x, end=" ")

0 1 2 3 4 5 6 7 8 9 

## Generators, yield
Generate the next item only when we need it

In [14]:
class Fib:
    def __init__(self, limit):
        self.limit = limit
    def __iter__(self):
        p0, p = 0, 1
        while p < self.limit:
            yield p
            p0, p = p, p + p0
        raise StopIteration

f100 = Fib(100)
for x in f100:
    print(x, end=" ")

1 1 2 3 5 8 13 21 34 55 89 

In [20]:
f10 = Fib(10)
i = iter(f10)
try:
    while 1:
        print(next(i), end=" ")
except:
    print("<= end")

1 1 2 3 5 8 <= end


## Coroutines (PEP-342)

In [39]:
def grep(pattern):
    print("Looking for %s" % pattern)
    while True:
        line = (yield)
        if pattern in line:
            print(line)

g = grep("edge")
next(g) # init
g.send("nothing sharp here")
g.send("EU_edge")

Looking for edge
EU_edge


## Operator overloading

In [43]:
class Fun:
    def __init__(self, function):
        self.function = function
    def __call__(self, *args, **kwargs):
        return self.function(*args, **kwargs)
    def __add__(self, funinst):
        def inner(*args, **kwargs):
            return funinst(self.function(*args, **kwargs))
        return Fun(inner)
    
f1 = Fun(lambda x,y: x+y)
print('f1', f1(1,2))
f2 = Fun(lambda x: x*x)
print('f2', f2(2))
f3 = f1 + f2
print('f3', f3(1,2))

f1 3
f2 4
f3 9


## `__getattr__`
Override attribute access

In [49]:
class JSObj(dict):
    def __getattr__(self, attr):
        return self.get(attr)
    def __setattr__(self, attr, value):
        self[attr] = value

o = JSObj({'a': 10, 'b': 'beka'})
print("o['a'] =", o['a'])
print("o.a =", o.a)
print("o.c =", o.c)
o.c = [1,2]
print("o.c =", o.c)

o['a'] = 10
o.a = 10
o.c = None
o.c = [1, 2]


## `__dict__`
*Direct* attribute access

`obj.attr = value === obj.__dict__['attr'] = value`

In [50]:
class Borg:
    __shared_state = {}
    def __init__(self):
        self.__dict__ = self.__shared_state
        
b1 = Borg()
b2 = Borg()
b1.answer = 42
print(b2.answer)

42


http://docs.python.org/reference/datamodel.html#special-method-names

## Nothing is private
*Do not do this ever*

In [52]:
def hide(o):
    class Proxy:
        __slots__ = () # only attributes listed in __slots__ can be accessed/defined
        def __getattr__(self, name):
            return getattr(o, name)
    return Proxy()

class HideMe:
    a = 42
    
a = hide(HideMe())
print(a.a)
a.a = 43

42


AttributeError: 'Proxy' object has no attribute 'a'

In [58]:
a.__getattr__.__closure__[0].cell_contents.a = 43
print(a.a)

43


## Decorators
- syntactic sugar
- `functool.wraps`
- `@classmethond`, `@staticmethod`

In [61]:
from functools import wraps

def logger(f):
    @wraps(f)
    def inner(*args, **kwargs):
        print("%s called with parameters: %s; %s" % (f.__name__, args, kwargs))
        retval = f(*args, **kwargs)
        print("%s returned: %s" % (f.__name__, retval))
        
        return retval
    return inner

@logger
def mul(a,b):
    return a*b
# ===> mul = logger(mul)

print(mul(2, b = 3))

mul called with parameters: (2,); {'b': 3}
mul returned: 6
6


## Descriptors
To "hide" getter/setter methods

- `__get__(self, instance, owner)`
- `__set__(self, instance, value)`
- `__delete__(self, instance)`

### Properties

In [65]:
class PropEx:
    __a = 1
    __b = 2
    
    @property
    def c(self):
        return self.__a + self.__b
    
    def getA(self):
        return self.__a
    def setA(self, value):
        self.__a = value
    def delA(self):
        self.__a = 1
        
    a = property(getA, setA, delA)
    # @property, @prop.setter, @prop.deleter

x = PropEx()
x.a = 12
print(x.c)
print(x.__class__.a.fset)

14
<function PropEx.setA at 0x7f10f0070378>


### Functions/methods
Functions are descriptors with `__get__`

In [73]:
def what_is_my_class(self):
    return self.__class__

class A:
    def method(self):
        pass

a = A()
print(a.method)
print(a.__class__.method) # different in Py2
print(what_is_my_class)

<bound method A.method of <__main__.A object at 0x7f10f000d208>>
<function A.method at 0x7f10f0070158>
<function what_is_my_class at 0x7f10f0070510>


In [72]:
a.wmc = what_is_my_class
print(a.wmc)
print(a.wmc())

<function what_is_my_class at 0x7f10f0070f28>


TypeError: what_is_my_class() missing 1 required positional argument: 'self'

In [71]:
a.wmc = what_is_my_class.__get__(a, A)
print(a.wmc)
print(a.wmc())

<bound method what_is_my_class of <__main__.A object at 0x7f10f0086e10>>
<class '__main__.A'>


## type()
- function: returns the type of the argument
- it's a type too, the root type of the classes/types (cause they are actually types)

In [74]:
print(type(object))
print(type(type(42)))
print(type(type))

<class 'type'>
<class 'type'>
<class 'type'>


- **AND** it can create new classes runtime

### Metaprograming

In [77]:
def constr(self):
    print("new instance created")
    self.foo = "foo"
    
def method(self):
    print("foo:", self.foo)
    print("PI:", self.PI)

# creating a new class = a simple function call
MyClass = type(
    "MyClass",              # classname
    (dict, object),         # baseclasses
    {                       # __dict__
        'PI': 3.14,
        'method': method,
        '__init__': constr
    }
)

myinstance = MyClass()
myinstance.method()

new instance created
foo: foo
PI: 3.14


### `__metaclass__`
When defined for a class, it is used instead of `type` at class generation

In [83]:
class mymeta(type):
    def __new__(mcs, name, bases, dict):
        dict['myprop'] = 'metaclass was here'
        return super(mymeta, mcs).__new__(mcs, name, bases, dict)

class MC(metaclass = mymeta):
    pass
    
mc = MC()
print(mc.myprop)

metaclass was here


You can do **anything** here, like type checking based on docstrings or annotations, decorating all the methods with caching/logging capabilities, ...

## Thank you!